In [1]:
import os
import torch
import sys, pathlib
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification,DebertaV2ForSequenceClassification, GPTNeoXForCausalLM
from llm_blender.pair_ranker.pairrm import DebertaV2PairRM
from transformers import DataCollatorWithPadding
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets, DatasetDict
import torch.nn.functional as F
from tqdm import tqdm
import re
import yaml

LOCAL_TRL_PARENT = "/workspace/Self_play_DRPO"
if LOCAL_TRL_PARENT not in sys.path:
    sys.path.insert(0, LOCAL_TRL_PARENT)

    
# now the import will use your local copy:
from trl import (
    DPOTrainer,
    DPOConfig,
    ModelConfig,
    DRPOTrainer,
    DRPOConfig,
)
from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE
from trl.data_utils import apply_chat_template
from trl.trainer.drpo_utils import PairRMPipeline

def process_split(original, seed = 42):
    swapped = original.map(lambda x: {
        'a1': x['a2'],
        'a2': x['a1'],
        # 'rank': 1 - int(random.random() < x['chosen_preference']),
        'rank': 1 - x['rank'],
    })

    return concatenate_datasets([original, swapped]).shuffle(seed=seed)

def load_model(model_path, task = 'generation', model_type = 'decoder', model_cache_path =  '/workspace/model_cache'):

    model_args = ModelConfig(model_path)
    model_torch_dtype = (model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype))
    model_kwargs = dict(
    revision = model_args.model_revision,
    torch_dtype = model_torch_dtype, 
    trust_remote_code = model_args.trust_remote_code,
    )

    padding_side = 'left' if model_type == 'decoder' else 'right'
    truncation_side = 'left' if model_type == 'decoder' else 'right'

    if task == 'generation':
        model_instance = AutoModelForCausalLM.from_pretrained(
            model_args.model_name_or_path,
            **model_kwargs,
            cache_dir = model_cache_path,
        )

    elif task == 'reward':
        model_instance = AutoModelForSequenceClassification.from_pretrained(
            model_args.model_name_or_path,
            **model_kwargs,
            cache_dir = model_cache_path,
        )
    

    model_tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path, 
        padding_side = padding_side, 
        truncation_side = truncation_side,
        use_fast = True,
        trust_remote_code = model_args.trust_remote_code,
        cache_dir = model_cache_path
    )

    if model_tokenizer.pad_token is None:
        model_tokenizer.pad_token = model_tokenizer.eos_token

    if getattr(model_instance.config, "pad_token_id", None) is None:
        model_instance.config.pad_token_id = model_tokenizer.pad_token_id

    if model_tokenizer.eos_token is None:
        model_tokenizer.eos_token = model_tokenizer.pad_token  

    if getattr(model_instance.config, "eos_token_id", None) is None:
        model_instance.config.eos_token_id = model_tokenizer.eos_token_id

    return model_instance, model_tokenizer



data_cache_path = "/workspace/dataset"
model_cache_path = '/workspace/model_cache'
ds_path = 'august66/drpo_ultrafeedback_qwen2.5-1.5b_first_iter_20k'
ref_policy_path = "Qwen/Qwen2.5-1.5B-Instruct" 
target_policy_path = "Qwen/Qwen2.5-1.5B-Instruct" 

#load training argument for drpo
with open("/workspace/Self_play_DRPO/self_play_drpo_code/training_config/config_normal_dist.yaml", "r") as f:
    training_args_config = yaml.safe_load(f)


/workspace/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 1234
ref_policy_model, ref_policy_tokenizer = load_model(ref_policy_path)
target_policy_model, target_policy_tokenizer = load_model(target_policy_path)
drpo_train = load_dataset(ds_path, cache_dir=data_cache_path, split = 'train')
drpo_train = process_split(drpo_train)
drpo_train = drpo_train.shuffle(seed=seed)

training_args = DRPOConfig(
    **training_args_config
)
preference_pipeline = PairRMPipeline(
    model_name_or_path = training_args.preference_model_id,
)

trainer = DRPOTrainer(
    model=target_policy_model,
    ref_model=ref_policy_model,
    preference_model=preference_pipeline,
    train_dataset = drpo_train,
    processing_class=ref_policy_tokenizer,
    args=training_args
)
trainer.train()

after chat template dataset sample: {'prompt': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nUtilizing the cognitive abilities of an artificial intelligence assistant has been found to provide substantial benefits in enhancing efficiency and productivity across various industries. In light of this, craft a compelling and original slogan that effectively captures the manifold advantages of leveraging AI assistance in the workplace, taking into account factors such as its ability to learn and adapt, its capacity for processing and analyzing vast amounts of data, and its potential to streamline and optimize complex workflows. Furthermore, the slogan should highlight the potential impact that AI assistants can have on achieving business objectives and improving overall performance, while also acknowledging the ethical considerations and potential challenges associated with their implementation.<|im_end|>\n<|im_start|>

Tokenizing train dataset: 100%|██████████| 40000/40000 [00:52<00:00, 767.54 examples/s]
wandb: Currently logged in as: ooooaugust (ooooaugust-london-school-of-economics-and-political-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{'dpo_reward_mean_ref': -0.128204345703125, 'dpo_reward_mean_target': -0.03230857849121094, 'standard deviation': 3.0, 'reward_a1': tensor([-0.1282], device='cuda:0', grad_fn=<SubBackward0>)}
{'ratio': 0.9994894862174988, 'numerator': 0.13291285932064056, 'denominator': 0.13298074901103973}


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{'dpo_reward_mean_ref': -0.11063957214355469, 'dpo_reward_mean_target': -0.11063957214355469, 'standard deviation': 3.0, 'reward_a1': tensor([0.4306], device='cuda:0', grad_fn=<SubBackward0>)}
{'ratio': 1.0, 'numerator': 0.13083387911319733, 'denominator': 0.13083387911319733}


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x731388154150>> (for post_run_cell), with arguments args (<ExecutionResult object at 731564f06d90, execution_count=2 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 731564f07690, raw_cell="seed = 1234
ref_policy_model, ref_policy_tokenizer.." transformed_cell="seed = 1234
ref_policy_model, ref_policy_tokenizer.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a22413130305f72756e706f64227d/workspace/Self_play_DRPO/self_play_drpo_code/drpo_normal_dist.ipynb#W3sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
drpo_train.select(range(1000))

Dataset({
    features: ['prompt', 'a1', 'a2', 'rank'],
    num_rows: 1000
})